In [355]:
import math
import os
import sys
sys.path.append("..")
import argparse
import string
import re

In [356]:
epsilon_hscan=['0.200000','0.300000','0.400000','0.500000','0.600000','0.700000','0.800000']
epsilon_scan=['0.2','0.3','0.4','0.5','0.6','0.7','0.8']
mu=['2','5','10','15']
# epsilon_hscan=['0.600000']
# epsilon_scan=['0.6']
# mu=['5']

dataset_file='/home/hnu/Disk0/llw/my_cpp12_hash/data/datasets/real2/threads-ask-ubuntu.hyp'
hscan_result_path='/home/hnu/Disk0/llw/my_cpp12_hash/output_lab1_csv/cluster_optimize_index_threads-ask-ubuntu'
scan_result_path='/home/hnu/Disk0/llw/pSCAN-master/datasets/threads-ask-ubuntu/result'
output_file='threads-ask-ubuntu.txt'

In [357]:
def read_graph(dataset_file):
    hyperedges=[]
    gs=0
    infile=open(dataset_file,'r')
    for line in infile:
        temp_string=line.strip('\n')
        temp=re.split(',',temp_string)
        e=[]
        for str_e in temp:
            if str_e!='':
                e.append(int(str_e))
                if gs<int(str_e):
                    gs=int(str_e)
        hyperedges.append(e)
    infile.close()
    print('load graph successfully, vertex num: '+str(gs+1)+' hyperedge num: '+str(len(hyperedges)))
    return hyperedges,gs+1

In [358]:
def construct_link_index(hyperedges,gs):
    vertex=[]
    for i in range(gs):
        vertex.append(set())
    for i in range(len(hyperedges)):
        for v in hyperedges[i]:
            vertex[v].add(i)
    print('construct link index successfully')
    return vertex

In [359]:
def load_hscan_cluster(hscan_result_file,hyperedges):
    hscan_ecluster=[]
    infile=open(hscan_result_file,'r')
    for line in infile:
        temp_string=line.strip('\n')
        temp=re.split(',',temp_string)
        e=[]
        for str_e in temp:
            if str_e!='':
                e.append(int(str_e))
        hscan_ecluster.append(e)
    infile.close()
    print('load the cluster result of HSCAN successfully, the cluster num: '+str(len(hscan_ecluster)))
    hscan_vcluster=[]
    for i in range(len(hscan_ecluster)):
        vs=set()
        for e in hscan_ecluster[i]:
            for v in hyperedges[e]:
                vs.add(v)
        hscan_vcluster.append(list(vs))
    return hscan_ecluster,hscan_vcluster

In [360]:
def load_scan_cluster(scan_result_file,hyperedges):
    scan_vcluster=[]
    cluster_id={}
    infile=open(scan_result_file,'r')
    new_id=0
    for line in infile:
        temp_string=line.strip('\n')
        if temp_string=='c/n vertex_id cluster_id':
            continue
        temp=re.split(' ',temp_string)
        v=int(temp[1])
        if temp[2] in cluster_id.keys():
            # print('cluster_id: '+temp[2]+' and new id : '+str(cluster_id[temp[2]]))
            scan_vcluster[cluster_id[temp[2]]].append(v)
        else:
            scan_vcluster.append([v])
            cluster_id[temp[2]]=new_id
            new_id+=1
    infile.close()
    print('load the cluster result of SCAN successfully, the cluster num: '+str(len(scan_vcluster)))
    scan_ecluster=[]
    for i in range(len(scan_vcluster)):
        vd={}
        for vertex in scan_vcluster[i]:
            vd[vertex]=1
        edge=set()
        for j in range(len(hyperedges)):
            flag=1
            for v in hyperedges[j]:
                if v not in vd.keys():
                    flag=0
                    break
            if flag==1 :
                edge.add(j)
        scan_ecluster.append(list(edge))
    return scan_ecluster,scan_vcluster

In [361]:
def compute_modularity(hyperedges,vertex,ecluster,vcluster):
    modularity=0
    connection_sum=0
    degree_sum=0
    
    connection_sum=len(hyperedges)
    for v_item in vertex:
        degree_sum+=len(v_item)
    
    overlap={}
    for c in vcluster:
        for v in c:
            if v in overlap.keys():
                overlap[v]+=1
            else:
                overlap[v]=1

    # print('dataset has '+str(connection_sum)+' hyperedge')
    # print('dataset the sum of d(v) is '+str(degree_sum))

    for i in range(len(vcluster)):
        cluster_connection=0
        cluster_degree=0
        cluster_connection=len(ecluster[i])
        # print('cluster '+str(i)+' has '+str(cluster_connection)+' hyperedges')
        left=cluster_connection

        for v in vcluster[i]:
            cluster_degree+=len(vertex[v])/overlap[v]

        # print('the sum of d(v) in cluster '+str(i)+' is '+str(cluster_degree))
 
        div=cluster_degree/degree_sum
        right=0
        for e in ecluster[i]:
            right+=pow(div,len(hyperedges[e]))
        modularity_cluster=left-right
        # print('modularity of cluster '+str(i)+' is '+str(modularity_cluster))
        modularity+=left-right

    return modularity/connection_sum

In [362]:
hyperedges,gs=read_graph(dataset_file)
vertex=construct_link_index(hyperedges,gs)
for i in range(len(epsilon_hscan)):
    for j in range(len(mu)):
        hscan_result_file=hscan_result_path+'_'+mu[j]+'_'+epsilon_hscan[i]+'.csv'
        scan_result_file=scan_result_path+'-'+epsilon_scan[i]+'-'+mu[j]+'.txt'
        hscan_ecluster,hscan_vcluster=load_hscan_cluster(hscan_result_file,hyperedges)
        scan_ecluster,scan_vcluster=load_scan_cluster(scan_result_file,hyperedges)
        modularity_hscan=compute_modularity(hyperedges,vertex,hscan_ecluster,hscan_vcluster)
        modularity_scan=compute_modularity(hyperedges,vertex,scan_ecluster,scan_vcluster)
        print('====================================')
        print('dataset: '+output_file+' epsilon: '+epsilon_scan[i]+' mu: '+mu[j])
        print('modularity(hscan): '+str(modularity_hscan))
        print('modularity(scan): '+str(modularity_scan))
        ofile=open(output_file,'a')
        ofile.write(' epsilon: '+epsilon_scan[i]+' mu: '+mu[j]+' modularity(hscan): '+str(modularity_hscan)+' modularity(scan): '+str(modularity_scan))
        ofile.write('\n')
        ofile.close()

load graph successfully, vertex num: 200975 hyperedge num: 192947
construct link index successfully
load the cluster result of HSCAN successfully, the cluster num: 3722
load the cluster result of SCAN successfully, the cluster num: 1985
dataset: threads-ask-ubuntu.txt epsilon: 0.2 mu: 2
modularity(hscan): 0.2553470031650159
modularity(scan): 0.2524872215011036
load the cluster result of HSCAN successfully, the cluster num: 107
load the cluster result of SCAN successfully, the cluster num: 1866
dataset: threads-ask-ubuntu.txt epsilon: 0.2 mu: 5
modularity(hscan): 0.21369681801276239
modularity(scan): 0.20195169550653627
load the cluster result of HSCAN successfully, the cluster num: 5
load the cluster result of SCAN successfully, the cluster num: 1272
dataset: threads-ask-ubuntu.txt epsilon: 0.2 mu: 10
modularity(hscan): 0.21187024977826152
modularity(scan): 0.11005464954592474
load the cluster result of HSCAN successfully, the cluster num: 3
load the cluster result of SCAN successfully